# Bibliothèques

In [1]:
import json
import pandas as pd

---

# Extraction des Data brutes

In [40]:
# Fonction pour extraire les données depuis le JSONL
def extract_data(file_path, max_rows=100000):
    data = []
    
    with open(file_path, 'r') as f:
        for idx, line in enumerate(f):
            if idx >= max_rows:
                break

            position = json.loads(line)
            fen = position.get("fen")

            for eval_data in position.get("evals", []):
                knodes = eval_data.get("knodes")
                depth = eval_data.get("depth")
                
                for pv in eval_data.get("pvs", []):
                    cp = pv.get("cp")
                    mate = pv.get("mate")
                    line = pv.get("line")

                    data.append({
                        "fen": fen,
                        "knodes": knodes,
                        "depth": depth,
                        "cp": cp,
                        "mate": mate,
                        "line": line
                    })

    return pd.DataFrame(data)

# Utilisation de la fonction
file_path = r'C:\Users\djiju\Documents\ML\data\lichess_db_eval.jsonl\lichess_db_eval.jsonl'  # Remplacez par le chemin de votre fichier
df_V1 = extract_data(file_path, max_rows=1000000)

# Affiche les premières lignes du DataFrame
#df_V1

KeyboardInterrupt: 

---

# Choix de la meilleur evaluation

Pour chaques positions, il plusieurs evaluations proposés par stockfish en fonction du nombre de noeuds et de la profondeur de recherche. Le resultat de ces evaluations donnes une "Ligne" c'est a dire l'enchainement de meilleurs coup optimums pour chaque joueurs. Par conséquent le meilleur coup est indiqué par le premier coup de la ligne.

| Type            | Description                                                                                                        |
|-----------------|--------------------------------------------------------------------------------------------------------------------|
| **knodes**      | Nombre de milliers de nœuds analysés par le moteur d'échecs pendant la recherche de la position.                   |
| **depth**       | Profondeur de la recherche, indiquant le nombre de coups (ou niveaux) explorés par le moteur.                     |
| **cp**          | Évaluation en centipions, représentant un avantage matériel en fonction du côté actif (positif pour Blancs, négatif pour Noirs). |
| **mate**        | Évaluation de mat. Si une valeur est donnée, elle indique le nombre de coups restants avant que le mat soit atteint. |


Parmis toutes les evaluations proposés pour une même position nous allons uniquement garder la meilleure pour créer notre base de donnée. Pour faire ce choix nous nous basons sur plusieurs critères :

1. **Profondeur de recherche (depth)** :
   - Plus la profondeur est grande, plus l'analyse est précise.
   - Priorisez les évaluations avec la profondeur maximale atteinte.

2. **Centipions (cp) ou mat (mate)** :
   - Si une évaluation donne un **mate**, elle est prioritaire, car un mat forcé est absolu.
   - En absence de **mate**, choisissez l'évaluation avec la valeur **cp** la plus proche de 0 si vous recherchez une position équilibrée, ou la plus élevée pour le joueur actif si vous cherchez un avantage.

3. **Nombre de nœuds analysés (knodes)** :
   - Si plusieurs évaluations ont la même profondeur, celle ayant exploré le plus grand nombre de nœuds est théoriquement plus fiable.

4. **Première variante principale (line)** :
   - En cas d’égalité sur les autres critères, utilisez une évaluation arbitraire ou choisissez selon des préférences spécifiques liées aux variantes.


In [298]:
def filter_best_evaluations(df):
    filtered_rows = []

    # Grouper par position FEN
    for fen, group in df.groupby('fen'):
        # Étape 1 : Sélectionner les évaluations avec la profondeur maximale
        max_depth = group['depth'].max()
        best_evals = group[group['depth'] == max_depth]
        
        # Étape 2 : Priorité à une évaluation de mat si elle existe
        if 'mate' in best_evals and best_evals['mate'].notna().any():
            best_row = best_evals.loc[best_evals['mate'].idxmin()]
        else:
            # Sinon, maximiser la valeur `cp`
            best_row = best_evals.loc[best_evals['cp'].idxmax()]
        
        # Ajouter la meilleure évaluation à la liste
        filtered_rows.append(best_row)
    
    # Créer un nouveau DataFrame avec les meilleures évaluations
    return pd.DataFrame(filtered_rows)

In [299]:
df_best_eval = filter_best_evaluations(df_V1)
df_best_eval

,fen,knodes,depth,cp,mate,line
38,6k1/4Rppp/8/8/8/8/5PPP/6K1 w - -,154,99,NaN,1.0,e7e8
22,6k1/6p1/8/4K3/4NN2/8/8/8 w - -,4300494,87,NaN,18.0,e4d6 g8h7 e5f5 g7g5 f4h5 h7h6 h5g3 h6g7 f5e6 g7f8
0,7r/1p3k2/p1bPR3/5p2/2B2P1p/8/PP4P1/3K4 b - -,200973,39,58.0,NaN,f7g7 e6e2 h8d8 e2d2 b7b5 c4e6 g7f6 e6b3 a6a5 a2a3
15,8/4r3/2R2pk1/6pp/3P4/6P1/5K1P/8 b - -,491568,58,0.0,NaN,e7a7 f2e3 a7a3 e3e4 a3a2 h2h4 g5h4 g3h4 a2h2 c6c1
33,r1b2rk1/1p2bppp/p1nppn2/q7/2P1P3/N1N5/PP2BPPP/...,72553,25,30.0,NaN,f1e1 c8d7 c1e3 d6d5 c4d5 e7a3 b2a3 a5c3 d5c6 d7c6


# Autres Méthodes 

On récupère le coup le plus joué

In [38]:
df_V1.to_csv('C:\\Users\\djiju\\Documents\\ML\\data\\test_df.csv', sep=';', encoding='utf-8', index=False)

In [39]:
df=pd.read_csv('C:\\Users\\djiju\\Documents\\ML\\data\\test_df.csv', sep=';')
df['line_first_move'] = df['line'].str[:4]
df_new=df.iloc[:, [0, -1]]
df_new

# Trouver le move le plus joué par FEN
most_played_moves = (
    df_new.groupby(["fen", "line_first_move"])
    .size()  # Compter les occurrences
    .reset_index(name="count")  # Renommer la colonne des comptes
    .sort_values(["fen", "count"], ascending=[True, False])  # Trier par FEN et par fréquence décroissante
    .drop_duplicates(subset="fen")  # Garder uniquement le premier (le plus fréquent) pour chaque FEN
)
# Conserver uniquement les colonnes fen et le move le plus fréquent
result = most_played_moves[["fen", "line_first_move"]]
# Afficher le résultat
print(result)
#df_new=df['fen','line_first_move']

                                                       fen line_first_move
0         1B1R4/4r1bk/1pb1B1pp/2p5/8/P1P3P1/5P1P/6K1 b - -            c6e8
3             1B1b4/2p2ppp/Bp2b3/3k4/1P6/P7/2K2PPP/8 b - -            d5c6
6             1B1b4/2p2ppp/Bp2b3/8/1P2k3/P7/2K2PPP/8 w - -            a3a4
11                    1B1b4/8/2pP2p1/5p2/k7/8/5PK1/8 b - -            d8b6
14                         1B1b4/8/8/8/4k1P1/6K1/8/8 w - -            b8a7
...                                                    ...             ...
3138376  rrrrkrrr/ppp1p1pp/8/5p2/4P3/5P2/PPP3PP/RRRRKRR...            d8d6
3138377  rrrrkrrr/ppppp1pp/8/5p2/4P3/3P4/PPP2PPP/RRRRKR...            d7d5
3138378  rrrrkrrr/ppppp1pp/8/5p2/8/3P4/PPP1PPPP/RRRRKRR...            f2f4
3138379  rrrrkrrr/pppppppp/8/8/8/3P4/PPP1PPPP/RRRRKRRR ...            f7f5
3138383  rrrrkrrr/pppppppp/8/8/8/8/PPPPPPPP/RRRRKRRR w ...            g2g4

[1000000 rows x 2 columns]


# Transformations des Données

In [300]:
import chess

### 1. Extraction de l'État de l'Échiquier à partir de la FEN

La colonne fen du dataframe contient la notation FEN (Forsyth-Edwards Notation), qui est utilisée pour décrire l'état d'un échiquier. Il faut convertir cette notation en une matrice 8x8 qui représente l'état de l'échiquier.

In [301]:
def fen_to_board_state(fen):
    board = chess.Board(fen)
    board_state = []
    for rank in range(8):
        row = []
        for file in range(8):
            piece = board.piece_at(chess.square(file, 7-rank))
            if piece:
                row.append(piece.piece_type if piece.color == chess.WHITE else -piece.piece_type)
            else:
                row.append(0)
        board_state.append(row)
    return board_state

### 2. Determination du Tour

La colonne Turn (White ou Black) correspond au joueur qui doit jouer. Si le FEN se termine par w, c'est au tour des blancs, sinon, c'est au tour des noirs.

In [302]:
def get_turn(fen):
    return "White" if fen.endswith('w') else "Black"

### 3. Calcul des Droits au Roque

Les droits au roque sont contenus dans le FEN, dans la partie juste avant le nombre de coups.
 Il y a 16 combinaisons possibles, qui sont donnée via le tableau ci-dessous :

| Notation | Description                          | Couleur  | Type de Roque           | Cases impliquées                  |
|----------|--------------------------------------|----------|-------------------------|-----------------------------------|
| K        | Petit roque côté roi possible        | Blancs   | Petit roque (côté roi)  | Roi : e1 → g1, Tour : h1 → f1    |
| Q        | Grand roque côté dame possible       | Blancs   | Grand roque (côté dame) | Roi : e1 → c1, Tour : a1 → d1    |
| k        | Petit roque côté roi possible        | Noirs    | Petit roque (côté roi)  | Roi : e8 → g8, Tour : h8 → f8    |
| q        | Grand roque côté dame possible       | Noirs    | Grand roque (côté dame) | Roi : e8 → c8, Tour : a8 → d8    |
| -        | Aucun roque possible                 | Blancs/Noirs | Aucun                | Aucun                             |

Exemple : **KQkq** -> Les deux camps peuvent roquer de chaque côté.


In [303]:
def get_castling(fen):
    return fen.split()[2]

### 4. Numéro du Coup

Les numéros de coups sont également dans la notation FEN.

In [304]:
def get_fullmove_number(fen):
    # Vérifier si la chaîne FEN a suffisamment de parties
    parts = fen.split()
    if len(parts) >= 6:
        return int(parts[5])
    else:
        # Retourner une valeur par défaut ou traiter le cas où le fullmove est absent
        return None  # ou une autre valeur selon votre besoin


### 5. Détermination des Coups Légaux et du Meilleur Coup

La colonne Legal Moves et Best Move nécessitent une analyse des coups légaux et du meilleur coup.

### 6. Pars Fen

La fonction parse_fen est utilisée pour convertir la chaîne FEN (Forsyth-Edwards Notation) en une matrice 8x8 qui représente l'état actuel de l'échiquier d'échecs

In [305]:
def parse_fen(fen):
    board_state = []
    rows = fen.split()[0].split('/')
    
    for row in rows:
        board_row = []
        for char in row:
            if char.isdigit():  # Si c'est un chiffre, on ajoute des cases vides
                board_row.extend([0] * int(char))
            else:
                # Convertir les pièces en valeurs numériques
                piece_values = {
                    'K': 6, 'Q': 5, 'R': 4, 'B': 3, 'N': 2, 'P': 1,
                    'k': -6, 'q': -5, 'r': -4, 'b': -3, 'n': -2, 'p': -1
                }
                board_row.append(piece_values.get(char, 0))
        board_state.append(board_row)
    
    return board_state

### 7. Assembler toutes les Colonnes

Une fois toutes les fonctions définies, il faut créer une nouvelle structure pour le dataframe. CAD itérer sur chaque ligne du dataframe actuel, appliquer ces transformations, puis créer les nouvelles colonnes. 

In [306]:
def transform_data(row):
    fen = row['fen']
    line = row['line']
    
    # Extraire les informations de FEN
    board_state = parse_fen(fen)  # Une fonction pour convertir la FEN en matrice 8x8
    turn = get_turn(fen)
    castling = get_castling(fen)
    fullmove_number = get_fullmove_number(fen)
    
    # Créer la ligne du dataframe avec les informations nécessaires
    return {
        'Board_State': board_state,
        'Turn': turn,
        'Castling': castling,
        'Fullmove Number': fullmove_number
    }

Appliquer les transformations

In [307]:
# Appliquer la transformation sur chaque ligne du DataFrame
df_cleaned = df_best_eval.apply(transform_data, axis=1, result_type='expand')

In [308]:
df_transformed = pd.DataFrame(df_cleaned.values.tolist(),columns=["Board_State", "Turn", "Castling", "Fullmove_Number"])
print(df_transformed.head(16))

                                         Board_State   Turn Castling  \
0  [[0, 0, 0, 0, 0, 0, -6, 0], [0, 0, 0, 0, 4, -1...  Black        -   
1  [[0, 0, 0, 0, 0, 0, -6, 0], [0, 0, 0, 0, 0, 0,...  Black        -   
2  [[0, 0, 0, 0, 0, 0, 0, -4], [0, -1, 0, 0, 0, -...  Black        -   
3  [[0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, -4, 0,...  Black        -   
4  [[-4, 0, -3, 0, 0, -4, -6, 0], [0, -1, 0, 0, -...  Black        -   

  Fullmove_Number  
0            None  
1            None  
2            None  
3            None  
4            None  
